### Having the data ready

In [3]:
import pandas as pd

mainData = pd.read_csv('I:\\Learnings\\Machine Learning\\Projects\\Kaggle\\Titanic\\Data\\train.csv')
features_considered = mainData[['PassengerId','Pclass','Sex','Age', 'SibSp','Parch', 'Survived']]
features_considered['family_members'] = features_considered['SibSp'] + features_considered['Parch']
features_considered.drop(['SibSp','Parch'], axis = 1)
features_considered['Age'].fillna(features_considered['Age'].mean(), inplace = True)

#matrix of features X
X = features_considered.iloc[:,[1,2,3,7]].values
#dependent variable vector y
y = features_considered.iloc[:,6].values

X.shape

F:\InstalledPrograms\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
F:\InstalledPrograms\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


(891, 4)

In [4]:
X

array([[3, 'male', 22.0, 1],
       [1, 'female', 38.0, 1],
       [3, 'female', 26.0, 0],
       ...,
       [3, 'female', 29.69911764705882, 3],
       [1, 'male', 26.0, 0],
       [3, 'male', 32.0, 0]], dtype=object)

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer([('encoder',OneHotEncoder(), [1])], remainder='passthrough')
X = transformer.fit_transform(X)
X = X[:,1:]

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state= 0,test_size=0.2)

In [136]:
#Tuning the model
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# define models and parameters
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.800972 using {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.787428 (0.044353) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.787428 (0.044353) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.788837 (0.043087) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.787428 (0.044353) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.787428 (0.044353) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.792117 (0.040520) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.789306 (0.041440) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.789306 (0.041440) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.800972 (0.042590) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.786039 (0.036634) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.786039 (0.036634) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.795827 (0.040612) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.742045 (0.036168) wit

In [137]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=0, C=1.0, penalty = 'l2',solver ='liblinear')
classifier.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [125]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X_train, y_train)


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [22]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv = 10)
print('Accuracy: {:.2f}%'.format(accuracies.mean()*100))

Accuracy: 80.21%


In [8]:
y_pred = classifier.predict(X_test)
#y_pred = regressor.predict(X_test)

In [9]:

# y_pred = 1 if y_pred >= 0.5 else 0
# for i in list(np.arange(y_pred.shape[0])):
#     y_pred[i] = 1 if y_pred[i] >= 0.5 else 0

# y_pred = y_pred.astype('int32')
y_pred

array([0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0], dtype=int64)

In [15]:
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
cm = confusion_matrix(y_test,y_pred)
#sns.heatmap(cm,annot=True)
print(cm)
accuracy_score(y_test, y_pred)

[[97 13]
 [19 50]]


0.8212290502793296

In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86       110
           1       0.79      0.72      0.76        69

    accuracy                           0.82       179
   macro avg       0.81      0.80      0.81       179
weighted avg       0.82      0.82      0.82       179



In [17]:
# Test on Actual test data
test_dataset = pd.read_csv('I:\\Learnings\\Machine Learning\\Projects\\Kaggle\\Titanic\\Data\\test.csv')
# test_dataset.head()
features_considered1 = test_dataset[['PassengerId','Pclass','Sex','Age', 'SibSp','Parch']]
features_considered1['family_members'] = features_considered1['SibSp'] + features_considered1['Parch']
features_considered1.drop(['SibSp','Parch'], axis = 1)
features_considered1['Age'].fillna(features_considered1['Age'].mean(), inplace = True)

#matrix of features X
X_actual_test = features_considered1.iloc[:,[1,2,3,6]].values

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer([('encoder',OneHotEncoder(), [1])], remainder='passthrough')
X_actual_test = transformer.fit_transform(X_actual_test)

X_actual_test = X_actual_test[:,1:]
X_actual_test



# y_actual_pred = regressor.predict(X_actual_test)
y_actual_pred = classifier.predict(X_actual_test)


# y_pred = 1 if y_pred >= 0.5 else 0
# for i in list(np.arange(y_actual_pred.shape[0])):
#     y_actual_pred[i] = 1 if y_actual_pred[i] >= 0.5 else 0

# y_actual_pred = y_actual_pred.astype('int32')

list(y_actual_pred)






F:\InstalledPrograms\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
F:\InstalledPrograms\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


[0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,


In [18]:
#get result dataframe

resultDf= pd.DataFrame({'Survived' : list(y_actual_pred)})
resultDf['PassengerId'] = features_considered1['PassengerId']

In [14]:
resultDf.to_csv('I:\\Learnings\\Machine Learning\\Projects\\Kaggle\\Titanic\\Data\\final_result4.csv')